In [2]:
print('=====================================================')
print('          Welcome to MovieMatch Recommender! ')
print('     Your personalized movie suggestions start here.')
print('          Rate a few movies to get started 🎥')
print('=====================================================')

          Welcome to MovieMatch Recommender! 
     Your personalized movie suggestions start here.
          Rate a few movies to get started 🎥
